# what will you buy if you can go back 5 years ago

In [2]:
import pandas as pd

sp500_companies = pd.read_csv('sp500_companies.csv')
sp500_index = pd.read_csv('sp500_index.csv')
sp500_stocks = pd.read_csv('sp500_stocks.csv')

print(sp500_companies.info())
print(sp500_index.info())
print(sp500_stocks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exchange             503 non-null    object 
 1   Symbol               503 non-null    object 
 2   Shortname            503 non-null    object 
 3   Longname             503 non-null    object 
 4   Sector               503 non-null    object 
 5   Industry             503 non-null    object 
 6   Currentprice         503 non-null    float64
 7   Marketcap            503 non-null    int64  
 8   Ebitda               474 non-null    float64
 9   Revenuegrowth        502 non-null    float64
 10  City                 503 non-null    object 
 11  State                483 non-null    object 
 12  Country              503 non-null    object 
 13  Fulltimeemployees    497 non-null    float64
 14  Longbusinesssummary  503 non-null    object 
 15  Weight               503 non-null    flo

In [5]:
# Convert date columns to datetime format for accurate merging
sp500_stocks['Date'] = pd.to_datetime(sp500_stocks['Date'])
sp500_index['Date'] = pd.to_datetime(sp500_index['Date'])

In [6]:
# Merge stock prices with fundamentals based on 'Symbol'
sp500_combined = pd.merge(
    sp500_stocks,  # Third DataFrame with stock price data
    sp500_companies,  # First DataFrame with company fundamentals
    on='Symbol', 
    how='left'
)

print(sp500_combined.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873172 entries, 0 to 1873171
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   Symbol               object        
 2   Adj Close            float64       
 3   Close                float64       
 4   High                 float64       
 5   Low                  float64       
 6   Open                 float64       
 7   Volume               float64       
 8   Exchange             object        
 9   Shortname            object        
 10  Longname             object        
 11  Sector               object        
 12  Industry             object        
 13  Currentprice         float64       
 14  Marketcap            int64         
 15  Ebitda               float64       
 16  Revenuegrowth        float64       
 17  City                 object        
 18  State                object        
 19  Country              

In [7]:
# Convert date columns to datetime format for merging
sp500_index['Date'] = pd.to_datetime(sp500_index['Date'])
sp500_combined['Date'] = pd.to_datetime(sp500_combined['Date'])

# Merge stock prices with S&P 500 index data based on 'Date'
sp500_full = pd.merge(
    sp500_combined,
    sp500_index,
    on='Date',
    how='left'
)

print(sp500_full.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873172 entries, 0 to 1873171
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   Symbol               object        
 2   Adj Close            float64       
 3   Close                float64       
 4   High                 float64       
 5   Low                  float64       
 6   Open                 float64       
 7   Volume               float64       
 8   Exchange             object        
 9   Shortname            object        
 10  Longname             object        
 11  Sector               object        
 12  Industry             object        
 13  Currentprice         float64       
 14  Marketcap            int64         
 15  Ebitda               float64       
 16  Revenuegrowth        float64       
 17  City                 object        
 18  State                object        
 19  Country              

In [16]:
import numpy as np
import cvxpy as cp

# Calculate daily returns for individual stocks
sp500_full['Daily_Return'] = sp500_full.groupby('Symbol')['Close'].pct_change(fill_method=None)

# Display rows with valid daily returns
print(sp500_full['Daily_Return'].isna().sum())

102129


In [25]:
# Filter out NaN values
df_returns = sp500_full[['Symbol', 'Date', 'Daily_Return']].dropna()

In [26]:
# Calculate mean returns and covariance matrix
mean_returns = df_returns.groupby('Symbol')['Daily_Return'].mean()

# Add stronger regularization to the diagonal
cov_matrix += np.eye(cov_matrix.shape[0]) * 1e-2

In [27]:
# Define variables for optimization
num_assets = len(mean_returns)
weights = cp.Variable(num_assets)
risk_tolerance = 0.5  # Adjustable based on user input

In [28]:
# Define the portfolio return and risk
portfolio_return = mean_returns.values @ weights
portfolio_risk = cp.quad_form(weights, cov_matrix.values)

In [29]:
# Define the optimization problem
problem = cp.Problem(
    cp.Maximize(portfolio_return - risk_tolerance * portfolio_risk),
    [cp.sum(weights) == 1, weights >= 0]
)

In [30]:
# Check for all-zero rows/columns
zero_rows = (cov_matrix.sum(axis=1) == 0).sum()
zero_cols = (cov_matrix.sum(axis=0) == 0).sum()

print(f"Number of zero rows: {zero_rows}, Number of zero columns: {zero_cols}")


Number of zero rows: 0, Number of zero columns: 0


In [31]:
from scipy.linalg import eigh

def nearest_psd(matrix):
    """Convert a covariance matrix to the nearest positive semi-definite matrix."""
    # Eigenvalue decomposition
    eigenvalues, eigenvectors = eigh(matrix)
    
    # Set negative eigenvalues to zero
    eigenvalues[eigenvalues < 0] = 0
    
    # Reconstruct the matrix
    psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    return psd_matrix

# Apply nearest positive semi-definite transformation
cov_matrix = nearest_psd(cov_matrix)

# Define the optimization problem again
portfolio_risk = cp.quad_form(weights, cov_matrix)

problem = cp.Problem(
    cp.Maximize(portfolio_return - risk_tolerance * portfolio_risk),
    [cp.sum(weights) == 1, weights >= 0]
)

# Solve the problem
problem.solve()

# Get optimal weights
optimal_weights = pd.Series(weights.value, index=mean_returns.index)
print("Optimal Portfolio Weights:", optimal_weights)

Optimal Portfolio Weights: Symbol
A      -1.417697e-21
AAPL   -1.254886e-21
ABBV   -1.370311e-21
ABNB   -9.542727e-22
ABT    -1.302443e-21
            ...     
XYL    -1.345258e-21
YUM    -1.390762e-21
ZBH    -1.394356e-21
ZBRA   -1.360080e-21
ZTS    -1.330089e-21
Length: 503, dtype: float64


In [38]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Define the objective function for portfolio optimization
def objective_function(weights):
    port_return = np.dot(mean_returns.values, weights)
    port_risk = np.dot(weights.T, np.dot(cov_matrix, weights))
    return -1 * (port_return - 0.5 * port_risk)

# Initial weights
num_assets = len(mean_returns)
init_weights = np.ones(num_assets) / num_assets  # Equally distributed weights

# Constraints: sum of weights = 1 and weights >= 0
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights must sum to 1
    {'type': 'ineq', 'fun': lambda x: x}  # Weights must be >= 0
]

# Bounds for weights (between 0 and 1)
bounds = [(0, 1) for _ in range(num_assets)]

# Solve using SciPy's minimize
result = minimize(objective_function, init_weights, bounds=bounds, constraints=constraints)

# Get optimized weights
optimal_weights = pd.Series(result.x, index=mean_returns.index)
print("Optimal Portfolio Weights:", optimal_weights)

Optimal Portfolio Weights: Symbol
A       8.029814e-18
AAPL    0.000000e+00
ABBV    0.000000e+00
ABNB    2.137832e-18
ABT     8.565970e-18
            ...     
XYL     0.000000e+00
YUM     2.874096e-18
ZBH     4.533162e-18
ZBRA    0.000000e+00
ZTS     0.000000e+00
Length: 503, dtype: float64


In [39]:
# Define a higher risk tolerance
risk_tolerance = 5.0  # Adjust to be more aggressive

# Redefine the objective function and solve again
def objective_function(weights):
    port_return = np.dot(mean_returns.values, weights)
    port_risk = np.dot(weights.T, np.dot(cov_matrix, weights))
    return -1 * (port_return - risk_tolerance * port_risk)

In [40]:
# Standardize the mean returns
mean_returns = (mean_returns - mean_returns.min()) / (mean_returns.max() - mean_returns.min())

# Normalize the covariance matrix
cov_matrix = (cov_matrix - cov_matrix.min().min()) / (cov_matrix.max().max() - cov_matrix.min().min())


In [41]:
min_weight = 0.01  # Minimum allocation of 1%

constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights must sum to 1
    {'type': 'ineq', 'fun': lambda x: x - min_weight}  # Weights must be >= min_weight
]


In [42]:
bounds = [(-0.1, 1) for _ in range(num_assets)]  # Allow up to 10% short-selling


In [43]:
from scipy.optimize import minimize

# Define the modified objective function
def modified_objective_function(weights):
    port_return = np.dot(mean_returns.values, weights)
    port_risk = np.dot(weights.T, np.dot(cov_matrix, weights))
    return -1 * (port_return - risk_tolerance * port_risk)

# Solve the optimization again
result = minimize(modified_objective_function, init_weights, bounds=bounds, constraints=constraints)

# Get new optimal weights
optimal_weights = pd.Series(result.x, index=mean_returns.index)
print("Optimal Portfolio Weights:", optimal_weights)


Optimal Portfolio Weights: Symbol
A       0.001988
AAPL    0.001988
ABBV    0.001988
ABNB    0.001988
ABT     0.001988
          ...   
XYL     0.001988
YUM     0.001988
ZBH     0.001988
ZBRA    0.001988
ZTS     0.001988
Length: 503, dtype: float64
